In [1]:
import os
from langchain.chat_models import init_chat_model
from langgraph.prebuilt import create_react_agent

API_KEY = "sk-123"

BASE_URL = "https://api.deepseek.com"

os.environ["OPENAI_API_KEY"] = API_KEY
os.environ["OPENAI_API_BASE"] = BASE_URL


llm = init_chat_model(model="openai:deepseek-chat",
                      disable_streaming=True,# 禁用流式传输
                      temperature=0.1,
                      max_tokens=10240
                      )


# 定义工具
from langchain_core.tools import tool


@tool("multiply_tool", parse_docstring=True)
def multiply(a: int, b: int) -> int:
    """Multiply two numbers.

    Args:
        a: First operand
        b: Second operand
    """
    return a * b



#您还可以使用 Pydantic 定义自定义输入架构：



from pydantic import BaseModel, Field

class MultiplyInputSchema(BaseModel):
    """Multiply two numbers"""
    a: int = Field(description="First operand")
    b: int = Field(description="Second operand")

@tool("multiply_tool", args_schema=MultiplyInputSchema)
def multiply_pydantic(a: int, b: int) -> int:
    return a * b





agent = create_react_agent(
    model=llm,
tools=[multiply],
)

agent.invoke(
    {"messages": [{"role": "user", "content": "what's 3 + 5 and 4 * 7?"}]}
)

{'messages': [HumanMessage(content="what's 3 + 5 and 4 * 7?", additional_kwargs={}, response_metadata={}, id='fceea18f-cb68-461a-a690-1dbcc6d52567'),
  AIMessage(content='3 + 5 equals **8**.\n\nFor 4 * 7, let me calculate that for you.', additional_kwargs={'tool_calls': [{'id': 'call_0_e9c90af1-c68c-41dc-be12-4612284ebe54', 'function': {'arguments': '{"a":4,"b":7}', 'name': 'multiply_tool'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 149, 'total_tokens': 195, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 149}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_8802369eaa_prod0425fp8', 'id': '87e4552d-220f-4338-ac35-ac635321d25d', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--93c67b97-d22e-407e-ad0b-02074b3953b8-0', tool_calls=[{'name': 'multiply_t

In [2]:
# 直接返回工具结果¶
# 用于立即返回工具结果并停止代理循环：return_direct=True

from langchain_core.tools import tool

@tool(return_direct=True)
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

agent = create_react_agent(
    model=llm,
    tools=[add]
)

agent.invoke(
    {"messages": [{"role": "user", "content": "what's 3 + 5?"}]}
)

{'messages': [HumanMessage(content="what's 3 + 5?", additional_kwargs={}, response_metadata={}, id='c9095131-05b8-4423-8c99-fa5a29f583d7'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_0_e3d3f882-752a-4df9-bba6-79cdc2047151', 'function': {'arguments': '{"a":3,"b":5}', 'name': 'add'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 121, 'total_tokens': 143, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 121}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_8802369eaa_prod0425fp8', 'id': '785c221b-bd94-48f8-9b68-7ef7838eb5ac', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--06a2b4b5-8aea-485f-9b62-515958aac8d5-0', tool_calls=[{'name': 'add', 'args': {'a': 3, 'b': 5}, 'id': 'call_0_e3d3f882-752a-4df9-bba6-79cdc2047151', 'type': '

In [5]:
# 强制使用工具¶
# 要强制代理使用特定工具，您可以在 ：tool_choicemodel.bind_tools()

import os
from langchain.chat_models import init_chat_model
from langgraph.prebuilt import create_react_agent

API_KEY = "sk-123"

BASE_URL = "https://api.deepseek.com"

os.environ["OPENAI_API_KEY"] = API_KEY
os.environ["OPENAI_API_BASE"] = BASE_URL


llm = init_chat_model(model="openai:deepseek-chat",
                      disable_streaming=True,# 禁用流式传输
                      temperature=0.1,
                      max_tokens=10240
                      )


from langchain_core.tools import tool

@tool(return_direct=True)
def greet(user_name: str) -> int:
    """Greet user."""
    return f"Hello {user_name}!"

tools = [greet]

llm.bind_tools(tools, tool_choice={"type": "tool", "name": "greet"})

agent = create_react_agent(
    model=llm,
    tools=tools
)

agent.invoke(
    {"messages": [{"role": "user", "content": "Hi, I am Bob"}]}
)

{'messages': [HumanMessage(content='Hi, I am Bob', additional_kwargs={}, response_metadata={}, id='36e5a489-a431-457f-9b6b-f10da043223d'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_0_f3aeca2e-8434-4224-a8c8-1d1a173a9f9d', 'function': {'arguments': '{"user_name":"Bob"}', 'name': 'greet'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 104, 'total_tokens': 124, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 104}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_8802369eaa_prod0425fp8', 'id': '0439d763-8d04-449b-baaf-753ebbbda378', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--7ee1aaa2-ddc7-4aea-81f0-4578c83bf5e6-0', tool_calls=[{'name': 'greet', 'args': {'user_name': 'Bob'}, 'id': 'call_0_f3aeca2e-8434-4224-a8c8-1d1a173a9f9

In [6]:
# 处理工具错误¶
# 默认情况下，代理将捕获工具调用期间引发的所有异常，并将这些异常作为工具消息传递给 LLM。要控制错误的处理方式，您可以通过其参数使用预构建的 ToolNode（在其中执行工具的节点）


from langgraph.prebuilt import create_react_agent

def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    if a == 42:
        raise ValueError("The ultimate error")
    return a * b

# Run with error handling (default)
agent = create_react_agent(
    model="openai:deepseek-chat",
    tools=[multiply]
)
agent.invoke(
    {"messages": [{"role": "user", "content": "what's 42 x 7?"}]}
)

{'messages': [HumanMessage(content="what's 42 x 7?", additional_kwargs={}, response_metadata={}, id='2f76cea0-a04a-4f63-9cf0-78e814babfc2'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_0_3e20f59c-8e73-4db7-afb1-0a9bf9abcf07', 'function': {'arguments': '{"a":42,"b":7}', 'name': 'multiply'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 123, 'total_tokens': 146, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 123}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_8802369eaa_prod0425fp8', 'id': '3c6dca32-12a1-4826-b419-2a9a852496db', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--0d66d28c-f0e0-4d92-b5dd-813435065e82-0', tool_calls=[{'name': 'multiply', 'args': {'a': 42, 'b': 7}, 'id': 'call_0_3e20f59c-8e73-4db7-afb1-0a9bf9abcf0